<a href="https://colab.research.google.com/github/ag4267research1/Solving-Poisson-s-Equation-with-GNO/blob/main/GNOPDE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/ag4267research1/Solving-Poisson-s-Equation-with-GNO.git

fatal: destination path 'Solving-Poisson-s-Equation-with-GNO' already exists and is not an empty directory.


In [ ]:
%cd Solving-Poisson-s-Equation-with-GNO

/content/Solving-Poisson-s-Equation-with-GNO


In [ ]:
!git config --global user.email "anishg12umbc.edu"
!git config --global user.name "ag4267research1"

!git add your_notebook.ipynb
!git commit -m "Add notebook"
!git push origin main

fatal: pathspec 'your_notebook.ipynb' did not match any files
On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean
fatal: could not read Username for 'https://github.com': No such device or address


In [17]:
!ls
!git add Solving_Poisson.ipynb


README.md
fatal: pathspec 'Solving_Poisson.ipynb' did not match any files
